<a href="https://colab.research.google.com/github/domingues100/IEEE---Water_Level/blob/main/Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# IMPORTS

In [ ]:
import os
from shutil import copy2
import csv
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PIL
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gc
import statistics
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics import classification_report
from keras.layers import Lambda
import tensorflow_hub as hub

# **KFOLD**

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/IC_Machine_Learning/instances_default.csv')

for ind in train_data.index:
  train_data.loc[ind, 'label'] = train_data.loc[ind, 'label'] - 3

train_data['label']=train_data["label"].values.astype(str)

Y = train_data[['label']]

In [ ]:
image_dir =  #image path

skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

num_epochs = 100

idg = ImageDataGenerator(width_shift_range=0.3,
                         height_shift_range=0.3,
                         rotation_range=0.4,
                         zoom_range=0.4,
                         fill_mode='nearest',
                         horizontal_flip = True,
                         rescale=1./255)

We don’t need to create X, because as mentioned in the documentation page for StratifiedKFold , it is sufficient to provide only the labels Y to generate the splits and hence we can put np.zeros(n_samples) instead of X.


In [ ]:
save_dir = save_path
fold_var = 1

n = Y.size

#when running for the first time, need to run this lines, it will give the folder to run for faster-rcnn, retinanet and regression.
#After the first time, comment this lines and uncomment lines bellow
for train_index, val_index in skf.split(np.zeros(n),Y):
  training_data = train_data.iloc[train_index]
  validation_data = train_data.iloc[val_index]


#lines to uncomment
# for i in range(1,6):
#   training_data = pd.read_csv(f"/content/drive/MyDrive/Transaciones/Classificação/training_data{i}.csv")
#   validation_data = pd.read_csv(f"/content/drive/MyDrive/Transaciones/Classificação/validation_data{i}.csv")


  training_data ['label']=training_data ["label"].values.astype(str)
  validation_data['label']=validation_data["label"].values.astype(str)

  training_data.to_csv(f"{save_dir}/training_data{fold_var}.csv") #saves training_data in a csv
  validation_data.to_csv(f"{save_dir}/validation_data{fold_var}.csv") #saves testing_data in a csv


  base_model = keras.applications.ResNet50V2(
               include_top=False,
               weights='imagenet',
               input_shape=(224, 224, 3))

  base_model.trainable = False

  model = keras.Sequential([

    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation ='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(5, activation='softmax')
  ])

  #data generators - training and validating
  train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir,
						             x_col = "id", y_col = "label",
						             class_mode = "categorical", shuffle = True, batch_size=32)

  valid_data_generator = idg.flow_from_dataframe(validation_data, directory = image_dir,
							           x_col = "id", y_col = "label",
							           class_mode = "categorical", shuffle = False, batch_size=32)

  # Optimizer
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

	# Compile Model
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', 'mse'])

	# Fit Model
  history = model.fit(train_data_generator, epochs=num_epochs, validation_data=valid_data_generator)
  # Save Model
  tf.keras.backend.clear_session()
  model.save(save_dir + f"/model{fold_var}.h5")
  fold_var += 1
  gc.collect()

# Load Model

Necessary run diretory cells and idg before



In [ ]:
from keras.models import load_model

base_model = keras.applications.ResNet50V2(
               include_top=False,
               weights='imagenet',
               input_shape=(224, 224, 3))

base_model.trainable = False


model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(256, activation ='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(5, activation='softmax')
  ])

In [ ]:
save_dir =  #save path
image_dir = #images

var = 1
accuracy = []
k=0

for k in range(5):

  #it will read the folders
  validation_data = pd.read_csv(f"{save_dir}/validation_data{k+1}.csv")

  validation_data['label']=validation_data["label"].values.astype(str)

  valid_data_generator = idg.flow_from_dataframe(validation_data, directory = image_dir,
							           x_col = "id", y_col = "label",
							           class_mode = "categorical", shuffle = False, batch_size=32)

  #it will load the model
  model.load_weights(f"{save_dir}/model{k+1}.h5")
  pred = model.predict(valid_data_generator)
  predicted = np.argmax(pred,axis=1)


  for itens in validation_data:
    val_data = validation_data["label"] .values.tolist()


  val_data = list(map(int, val_data))
  print(val_data)
  j=0
  resultado = 0
  for elements in val_data:
    if val_data[j] == predicted[j]:
      resultado += 1
    j += 1

  accuracy.append(resultado/len(val_data))
  k += 1

  dados = {"predicted": predicted, "label": val_data}

  df = pd.DataFrame(dados)
  df.to_csv(f"{save_dir}/df{k}.csv")
